In [18]:
from glove import Corpus, Glove
import pandas as pd
import numpy as np

# l1='FC699'
l1='1189'
loc = l1+'/'+l1+'_SeqCode_withthresholdC.csv'
data = pd.read_csv(loc)

dim=400
dataset=[]
Feature_Type=1

if Feature_Type == 1:
    data['features'] = data['Sequence']
    for z in data['features']:
        dataset.append(list(z))
elif Feature_Type == 2: 
    data['features'] = data['Seq-Code']
    for z in data['features']:
        dataset.append(list(z))
else:
    data['features'] = data['Str-Code']
    for z in data['features']:
        dataset.append(z.split("--"))
        
corpus = Corpus()
corpus.fit(dataset, window = 1)
glove = Glove(no_components = dim, learning_rate = 0.01)
glove.fit(matrix = corpus.matrix, epochs = 30, no_threads = 4, verbose = False)
glove.add_dictionary(corpus.dictionary)
# glove.save('glove_model')
# glove.load('glove_model')

def buildWordVector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    
    if Feature_Type == 1 or Feature_Type == 2:
        wrds = tokens
    else:
        wrds = tokens.split("--")
        
    for word in wrds:
        try:
            vec += glove.word_vectors[glove.dictionary[word]].reshape((1, size))
            count += 1.
        except KeyError: # handling the case where the token is not
                         # in the corpus. useful for testing.
            continue
    if count != 0:
        vec /= count
    return vec
from sklearn.preprocessing import scale
feat_glove = np.concatenate([buildWordVector(z, dim) for z in data['features']])
feat_glove = scale(feat_glove)

df=pd.DataFrame(feat_glove)
df['PDBid'] = data['PDBid']
df['Class'] = data['Class']

if Feature_Type == 1:
    df.to_csv(l1+'/CE_Seq_'+l1+'.csv')
elif Feature_Type == 2:     
    df.to_csv(l1+'/CE_Str_'+l1+'.csv')
else:    
    df.to_csv(l1+'/CE_StrCode_'+l1+'.csv')    